In [1]:
# Load EDA
import numpy as np
import pandas as pd 
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

# other packages
import nltk
import string
import ast
import re
import unidecode
import unicodedata

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from collections import Counter

import pickle 
import unidecode, ast
from pathlib import Path


In [2]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# that's all the libaries that we need ..........

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sangames/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sangames/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sangames/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sangames/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# path to all files
DATA_PATH = "./data/updated_saree_data_v2.csv"
CLEAN_PATH = "./data/data_parsed_new.csv"
TFIDF_ENCODING_PATH = "./model/data_tfidf_encodings.pkl"
TFIDF_MODEL_PATH = "./model/data_tfidf.pkl"

#Initialising stopwords for english
stop_words = set(stopwords.words('english'))

# Load Our Dataset
def loadData(fileName): 
    df = pd.read_csv(fileName)
    return df 

#save the file
def saveData(df, fileName): 
    df.to_csv(fileName, index=False)  
    

def parseDataFile():
    
    # parses the data into words
    rec_df = loadData(DATA_PATH)

    # change the way the sentence is arranged in the data
    #rec_df['ingredients'] = rec_df['ingredients'].map(str) + ',' + rec_df['recipe_name'].map(str)
    rec_df['desc_temp'] = rec_df['desc'].str.split()

    rec_df['keywords'] = rec_df['desc_temp'].apply(lambda x: ingredient_parser(x))

    df = rec_df[['id', 'title','desc', 'keywords', 'img_url', 'found']]
    df = rec_df.dropna()

    saveData(df,CLEAN_PATH)

def createModel():
   
    # load in parsed recipe dataset     
    df_rec = loadData(CLEAN_PATH)
    df_rec['keywords'] = df_rec.desc.values.astype('U')

    # TF-IDF feature extractor 
    tfidf = TfidfVectorizer()
    tfidf.fit(df_rec['keywords'])
    tfidf_recipe = ve???? check code
    
    # save the tfidf model and encodings 
    with open(TFIDF_MODEL_PATH, "wb") as f:
        pickle.dump(tfidf, f)

    with open(TFIDF_ENCODING_PATH, "wb") as f:
        pickle.dump(tfidf_recipe, f)
        

In [4]:
def title_parser(title):
    title = unidecode.unidecode(title)
    return title 

def ingredient_parser(ingreds):
    
    #showStatus("ingredient parser")
    user_selected_words_to_remove = ['(',')','.','\'','saree','matching', 'lehenga','women', 'woman','shubh','self','fresh','a', 'and',  'or', 'large', 'extra', 'free', 'small', 'from', 'higher', 'for', 'the', 'plain', 'plus' ]
    # The ingredient list is now a string so we need to turn it back into a list. We use ast.literal_eval
    if isinstance(ingreds, list):
        ingredients = ingreds
    else:
        ingredients = ast.literal_eval(ingreds)

    # We first get rid of all the punctuation. We make use of str.maketrans. It takes three input 
    # arguments 'x', 'y', 'z'. 'x' and 'y' must be equal-length strings and characters in 'x'
    # are replaced by characters in 'y'. 'z' is a string (string.punctuation here) where each character
    #  in the string is mapped to None. 
    translator = str.maketrans('', '', string.punctuation)
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for i in ingredients:
        i.translate(translator)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', i)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]      
        # remove stop words
        items = [word for word in items if word not in stop_words]        
        # remove accents
        items = [unidecode.unidecode(word) for word in items] #''.join((c for c in unicodedata.normalize('NFD', items) if unicodedata.category(c) != 'Mn'))
        items = [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore') for word in items]        
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # Get rid of other user selected words
        items = [word for word in items if word not in user_selected_words_to_remove]
        # remove all square brackets
        items = [remove_between_square_brackets(word) for word in items]
        # remove all special characters
        items = [remove_special_characters(word) for word in items]
        if items:
            ingred_list.append(' '.join(items)) 
    ingred_list = " ".join(ingred_list)
    return ingred_list

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


In [5]:
parseDataFile()

In [6]:
createModel()
    